In [1]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm



import numpy as np
import os
import tensorflow as tf
import shutil


# from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers, initializers, regularizers, metrics
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1876563945833553002
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4986830848
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10891931645649765409
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1660 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [4]:
from io import BytesIO
import time, random
from tqdm import tqdm

from PIL import Image

In [8]:
BASE_DIR = 'D:/original_image/'
names = ["63building", "castle","general","indep_door","judgement_castle","lotte_tower","namsan","tapgol_park","seoulstation",\
        "coex","moonlight"]

tf.random.set_seed(1)

In [9]:
if not os.path.isdir(BASE_DIR + 'train/'):
    for name in names:
        os.makedirs(BASE_DIR + 'train/' + name)
        os.makedirs(BASE_DIR + 'val/' + name)
        os.makedirs(BASE_DIR + 'test/' + name)
        

In [10]:
orig_folders = ["/63building/", "/castle/","/general/", "/indep_door/","/judgement_castle/", "/lotte_tower/","/namsan/", "/tapgol_park/",\
               "/seoulstation/","/coex/","/moonlight/"]

            

In [11]:
for folder_idx, folder in enumerate(orig_folders):
    files = os.listdir(BASE_DIR + folder)
    number_of_images = len([name for name in files])
    n_train = int((number_of_images * 0.6) + 0.5)
    n_valid = int((number_of_images* 0.25) + 0.5)
    n_test = number_of_images - n_train - n_valid
    print(number_of_images, n_train, n_valid, n_test)
    for idx, file in enumerate(files):
        file_name = BASE_DIR + folder + file
        if idx < n_train:
            shutil.move(file_name, BASE_DIR + "train/" + names[folder_idx])
        elif idx < n_train + n_valid:
            shutil.move(file_name, BASE_DIR + "val/" + names[folder_idx])
        else:
            shutil.move(file_name, BASE_DIR + "test/" + names[folder_idx])

0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0


In [12]:
train_gen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
valid_gen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_gen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

#train에 저장
train_batches = train_gen.flow_from_directory(
    'D:/original_image/train',
    target_size = (240, 240),
    class_mode = 'sparse',
    batch_size = 2,
    shuffle=True,
    color_mode="rgb",
    classes=names
)

#val에 저장
val_batches = valid_gen.flow_from_directory(
    'D:/original_image/val',
    target_size = (240, 240),
    class_mode = 'sparse',
    batch_size = 2,
    shuffle=False,
    color_mode="rgb",
    classes=names
)

#테스트에 저장
test_batches = valid_gen.flow_from_directory(
    'D:/original_image/test',
    target_size = (240, 240),
    class_mode = 'sparse',
    batch_size = 2,
    shuffle=False,
    color_mode="rgb",
    classes=names
)

Found 6600 images belonging to 11 classes.
Found 2750 images belonging to 11 classes.
Found 1650 images belonging to 11 classes.


In [13]:
train_batch = train_batches[0]
print(train_batch[0].shape)
print(train_batch[1])

test_batch = test_batches[0]
print(test_batch[0].shape)
print(test_batch[1])

(2, 240, 240, 3)
[2. 9.]
(2, 240, 240, 3)
[0. 0.]


In [14]:
model = keras.models.Sequential()
model.add(layers.Conv2D(32, (3,3), strides=(1,1), padding="valid", activation='relu', input_shape=(240,240,3)))
model.add(layers.MaxPool2D(2,2))

In [15]:
# vgg 모델
vgg_model = tf.keras.applications.vgg16.VGG16()
print(type(vgg_model))
vgg_model.summary()

<class 'tensorflow.python.keras.engine.training.Model'>
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block

In [16]:
model = tf.keras.models.Sequential()
for layer in vgg_model.layers[0:-1]:
    model.add(layer)

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [18]:
# 마지막 레이어만 train해주면됨
#앞에 레이어 non trainable로 바꿔줌
for layer in model.layers:
    layer.trainable = False
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [19]:
model.add(layers.Dense(11))

In [20]:
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim = keras.optimizers.Adam(lr=0.001)
metrics = ["accuracy"]

model.compile(optimizer=optim, loss=loss, metrics=metrics)

In [21]:
preprocess_input = tf.keras.applications.vgg16.preprocess_input

In [22]:
train_gen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)
valid_gen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)
test_gen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)

train_batches = train_gen.flow_from_directory(
    'D:/original_image/train',
    target_size = (240, 240),
    class_mode = 'sparse',
    batch_size = 2,
    shuffle=True,
    color_mode="rgb",
    classes=names
)

val_batches = valid_gen.flow_from_directory(
    'D:/original_image/val',
    target_size = (240, 240),
    class_mode = 'sparse',
    batch_size = 2,
    shuffle=False,
    color_mode="rgb",
    classes=names
)

test_batches = valid_gen.flow_from_directory(
    'D:/original_image/test',
    target_size = (240, 240),
    class_mode = 'sparse',
    batch_size = 2,
    shuffle=False,
    color_mode="rgb",
    classes=names
)

Found 6600 images belonging to 11 classes.
Found 2750 images belonging to 11 classes.
Found 1650 images belonging to 11 classes.


In [ ]:
# folder = "D:/original_image/test/moonlight"
# file_list = [x for x in os.listdir(folder)]

In [ ]:
# file_list

In [ ]:
# train_list=[]
# for i in tqdm(range(len(file_list))):
#     try:        
#         image = Image.open("D:/original_image/test/moonlight/" + file_list[i])
#         image.save("D:/error_image/" + str(i) + '.jpg')
#         train_list.append(image)
#     except:
#         print('error!!',file_list[i])

In [26]:
epochs = 15

early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5
)

history = model.fit(train_batches, validation_data=val_batches,
         callbacks=[early_stopping],
         epochs=epochs)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 3300 steps, validate for 1375 steps
Epoch 1/15
3300/3300 [==============================] - 99s 30ms/step - loss: 0.0064 - accuracy: 0.9994 - val_loss: 0.0284 - val_accuracy: 0.9949
Epoch 2/15
3300/3300 [==============================] - 100s 30ms/step - loss: 0.0096 - accuracy: 0.9976 - val_loss: 0.0360 - val_accuracy: 0.9964
Epoch 3/15
3300/3300 [==============================] - 101s 31ms/step - loss: 5.7590e-07 - accuracy: 1.0000 - val_loss: 0.0352 - val_accuracy: 0.9964
Epoch 4/15
3300/3300 [==============================] - 101s 31ms/step - loss: 3.1148e-07 - accuracy: 1.0000 - val_loss: 0.0337 - val_accuracy: 0.9964
Epoch 5/15
3300/3300 [==============================] - 102s 31ms/step - loss: 1.2816e-07 - accuracy: 1.0000 - val_loss: 0.0299 - val_accuracy: 0.9967
Epoch 6/15
3300/3300 [==============================] - 101s 31ms/step - loss: 7.0441e-09 - accuracy: 1.0000 - val_loss: 0.0285 - val_accuracy: 0.9971


In [27]:
loss, acc = model.evaluate(test_batches)
acc

  ...
    to  
  ['...']
825/825 [==============================] - 19s 23ms/step - loss: 5.3765e-04 - accuracy: 0.9994


0.99939394

In [46]:
model.save('model')

INFO:tensorflow:Assets written to: model\assets


In [47]:
new_model = load_model('model')

In [49]:
new_model.evaluate(test_batches)

825/825 [==============================] - 19s 23ms/step - loss: 5.3765e-04 - accuracy: 0.9994


[0.0005376493907373391, 0.99939394]